In [1]:
# input data explorer
import os, sys
import numpy as np
import tensorflow as tf

# configure Tensorflow
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf.gpu_options.allow_growth=True

# intialize TF
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
init = tf.global_variables_initializer()
sess.run(init)

# iterate through data 

/home/yoyoteng/anaconda3/envs/py35tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yoyoteng/anaconda3/envs/py35tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yoyoteng/anaconda3/envs/py35tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yoyoteng/anaconda3/envs/py35tf/lib

In [2]:
# import stuff.
from fcvaegan import (
    Model,
    MODEL_DIR,
    INPUT_TENSOR_NAME,
    SIGNATURE_NAME,
    read_and_decode,
    PARAMS,
    W,H,C,CY,
    TRAIN_DIR,
    NUM_EXAMPLES_TRAIN,
    NUM_EXAMPLES_VALIDATION,
    NUM_EXAMPLES_TEST,
)
params = dict(PARAMS)
params.update({
    'is_training':True, # this impact inferences....due to batch norm...?
    'batch_size':4, # <<--- 4 is good for 6gb gpu.
})

batch_size = 1

def _input_fn(training_dir, training_filename, batch_size=batch_size):
    test_file = os.path.join(training_dir, training_filename)
    filename_queue = tf.train.string_input_producer([test_file])

    image, label = read_and_decode(filename_queue)
    images, labels = tf.train.batch(
        [image, label], batch_size=batch_size,
        capacity=1000 + 10 * batch_size)
    return {INPUT_TENSOR_NAME: images}, labels


#def eval_input_fn(training_dir=TRAIN_DIR, batch_size=batch_size, params=None):
#    return _input_fn(training_dir, 'validation.tfrecords', batch_size=batch_size)
#tf_images,tf_labels = eval_input_fn(batch_size=batch_size)

def train_input_fn(training_dir=TRAIN_DIR, batch_size=batch_size, params=None):
    return _input_fn(training_dir, 'train.tfrecords', batch_size=batch_size)
tf_images,tf_labels = train_input_fn(batch_size=batch_size)





Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.


Instructions for updating:
Queu

In [3]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord,sess=sess)    


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import yaml
with open('/media/external/scisoft/fc-vae-gan/data/label.yml','r') as f:
    label_dict = yaml.load(f.read())
max_labe_num = len(label_dict)

In [6]:
total_count,bins = np.histogram([],bins=max_labe_num,range=(0,max_labe_num))
total_count = np.array(total_count)
for n in range(NUM_EXAMPLES_TRAIN):
    img, lbl = sess.run([tf_images,tf_labels])
    count,bins = np.histogram(list((lbl.ravel())),bins=max_labe_num,range=(0,max_labe_num))
    total_count += np.array(count)

In [10]:
for ind in np.argwhere(total_count>np.percentile(total_count,95)):
    print(label_dict[ind[0]],total_count[ind[0]])

aeroplane 774745
building 4636206
bus 1051391
car 1802100
cat 2348562
chair 1181837
dog 2080085
fence 833917
floor 2246440
grass 4786213
ground 4621157
horse 679395
motorbike 840048
person 5878656
road 1761354
sky 6455033
sofa 1583421
table 1169882
train 978967
tree 4343756
unknown 2412597
wall 4506981
water 2329228


In [8]:
total_count

array([      0,     269,  774745,    1763,     108,     550,    2080,
          2237,   21761,   66987,    1476,    5234,       0,    1368,
           182,   26156,    1153,    2341,  259666,  542774,       0,
           207,   67328,  630591,       0,  525907,   70406,   12816,
         13725,    4172,   91288,  602538,     898,  123708,  465762,
           105,   27841,  149963,       0,    2182,   18687,     144,
         90245,   23693, 4636206, 1051391,  579991,       0,   62231,
             0,       0,    2968,       0,    1602,       0,   18724,
          2833,    2754,     626, 1802100,    2386,    8964,   26853,
             0,       0, 2348562,   16711,     230,  471751,    1417,
          6395,     581, 1181837,     461,       0,     806,      78,
             0,    6564,       0,  519946,       0,     155,     460,
           388,   43271,    3417,    3359,   25193,       0,    1506,
             0,       0,       0,       0,       0,   22688,       0,
        418361,     

In [45]:
STOP = False
if STOP:
    coord.request_stop()
    # Wait for threads to stop
    coord.join(threads)